In [1]:
import pandas as pd
import os
import gc
from tqdm import tqdm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
os.getcwd()

'C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\Program'

In [2]:
df = pd.read_csv('kdd/train_test_folder/train_kdd/train.csv')
traindf = pd.read_csv('./kdd/dataset/mix_dataset/neptune.csv')

X_main = df.drop('label', axis=1)
y_main = df['label']

train_test_index = [df['Unnamed: 0'], df['Unnamed: 0']]


In [42]:
from multiprocessing import Pool, cpu_count

window_size = 64
batch_size = 1
epochs = 10
n_features = X_main.shape[1]

DL_args = [window_size, batch_size, epochs]

chunk_size = 200
X = traindf.drop('label', axis=1)
y = traindf['label']

Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)


In [43]:
def sequential_models(window_size, n_features):

    models = {
        'LSTM' : lstm(window_size, n_features)
    }

    return models

def lstm(window_size, n_features):
    model = Sequential()
    model.add(LSTM(512, input_shape=(window_size, n_features), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = RMSprop(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model
models = sequential_models(window_size, n_features)

In [44]:
def process_data(Data, train_index, test_index, batch_size, window_size, chunk_size=200):
    def rearrange_sequences(generator, index, batch_size, window_size):
        rearranged_data = []

        for idx in tqdm(index, desc="Processing sequences"):
            if idx >= window_size - 1:  # Ensure index has enough preceding samples for a full sequence
                sequence_end = idx + 1  # Sequence ends at the current index (inclusive)
                sequence_start = sequence_end - window_size  # Sequence starts 'window_size' samples before the end

                batch_x = generator.data[sequence_start:sequence_end]  # Extract feature sequence
                batch_y = generator.targets[idx]  # Corresponding label

                rearranged_data.append((batch_x, batch_y))

        return rearranged_data
    # Process training data
    print('Training data processing...')
    train_data = rearrange_sequences(Data, train_index, batch_size, window_size)
    gc.collect()
    
    print('Testing data processing...')
    test_data = rearrange_sequences(Data, test_index, batch_size, window_size)
    del rearrange_sequences
    gc.collect()
    return train_data, test_data
    
def separate_features_labels(data, window_size):
    features = np.array([item[0] for item in data], dtype=np.float32)  # item[0] is each sequence
    labels = np.array([item[1] for item in data], dtype=np.float32)  # item[1] is the corresponding label

    # Ensure features are reshaped to (number_of_sequences, window_size, number_of_features_per_timestep)
    features = features.reshape(-1, window_size, features.shape[-1])

    return features, labels

def training_DL(models, df, DL_args, train_test_index):
    results = {}
    X = df.drop('label', axis=1)
    y = df['label']
    
    window_size, batch_size, epochs = DL_args
    
    Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)
    print(f'Data type {type(Data)}')
    train_index, test_index = train_test_index
    print(train_index)
    print(test_index)
    print('Processing Training data...')
    
    train_data, test_data = process_data(Data, train_index, test_index, batch_size, window_size)
    gc.collect()
    X_train, y_train = separate_features_labels(train_data, window_size)
    X_test, y_test = separate_features_labels(test_data, window_size)
    return X_train, X_test, y_train, y_test

In [45]:
Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)
print(f'Data type {type(Data)}')
train_index, test_index = train_test_index
print('Processing Training data...')

X_train, X_test, y_train, y_test = training_DL(models, traindf, DL_args, train_test_index)


Data type <class 'keras.preprocessing.sequence.TimeseriesGenerator'>
Processing Training data...
Data type <class 'keras.preprocessing.sequence.TimeseriesGenerator'>
0          90597
1          89016
2         106202
3          74603
4         104247
           ...  
103956     38990
103957    122689
103958    126235
103959     10237
103960     94409
Name: Unnamed: 0, Length: 103961, dtype: int64
0          90597
1          89016
2         106202
3          74603
4         104247
           ...  
103956     38990
103957    122689
103958    126235
103959     10237
103960     94409
Name: Unnamed: 0, Length: 103961, dtype: int64
Processing Training data...
Training data processing...


Processing sequences: 100%|██████████| 103961/103961 [00:02<00:00, 39474.03it/s]


Testing data processing...


Processing sequences: 100%|██████████| 103961/103961 [00:02<00:00, 36335.76it/s]


In [46]:
# window_size = 64
# batch_size = 4
# epochs = 10
X_train[0][63]

array([0.0000000e+00, 2.5362983e-08, 9.9241383e-08, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.9138943e-03,
       1.9569471e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 5.0000000e-01, 1.0000000e+00, 0.0000000e+00,
       7.8039217e-01, 3.1372551e-02, 3.9999999e-02, 3.9999999e-02,
       9.9999998e-03, 2.5000000e-01, 9.9999998e-03, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [51]:
train_test_index[0][0]

90597

In [47]:
X_train.shape

(103925, 64, 122)

In [71]:
y_train.shape

(103925,)

In [49]:
Data[0][0]

array([[[0.00000000e+00, 3.55806412e-07, 0.00000000e+00, ...,
         0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.05799870e-07, 0.00000000e+00, ...,
         0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.24643560e-07, 2.13292635e-06, ...,
         0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]])

In [21]:
y_train[0]

0.0

In [39]:
y

0         0
1         0
2         1
3         0
4         0
         ..
148512    0
148513    0
148514    0
148515    0
148516    0
Name: label, Length: 148517, dtype: int64

In [66]:
sequence_start_index = train_test_index[0][1] - (window_size - 1)
sequence_start_index

88953

In [67]:
batch_index = sequence_start_index // batch_size
batch_index

88953

In [68]:
position_within_batch = sequence_start_index % batch_size
position_within_batch

0

In [69]:
# Original data point
array = traindf.drop('label', axis=1).iloc[train_test_index[0][1]].to_numpy()

# Data from the generator
datagen = Data[batch_index][0][0][window_size-1]

# Comparison
equal = np.array_equal(array, datagen)
equal

In [17]:
import glob
import os
files = glob.glob(f'kdd/Training/model/**', recursive=True)

In [25]:
files = glob.glob(f'kdd/Training/model/**', recursive=True)
for file in files:
    if 'LSTM' in file:
        text1 = file.split('/')
        text2 = file.split('\\')
        model = text2[1]
        print(model)
        newname = file.replace('LSTM',f'{model}_LSTM_model')
        print(newname)
        os.rename(file,newname)
        print(f'Changing {file} with {newname}')

apache2
kdd/Training/model\apache2\apache2_LSTM_model.h5
Changing kdd/Training/model\apache2\LSTM.h5 with kdd/Training/model\apache2\apache2_LSTM_model.h5
back
kdd/Training/model\back\back_LSTM_model.h5
Changing kdd/Training/model\back\LSTM.h5 with kdd/Training/model\back\back_LSTM_model.h5
buffer_overflow
kdd/Training/model\buffer_overflow\buffer_overflow_LSTM_model.h5
Changing kdd/Training/model\buffer_overflow\LSTM.h5 with kdd/Training/model\buffer_overflow\buffer_overflow_LSTM_model.h5
ftp_write
kdd/Training/model\ftp_write\ftp_write_LSTM_model.h5
Changing kdd/Training/model\ftp_write\LSTM.h5 with kdd/Training/model\ftp_write\ftp_write_LSTM_model.h5
guess_passwd
kdd/Training/model\guess_passwd\guess_passwd_LSTM_model.h5
Changing kdd/Training/model\guess_passwd\LSTM.h5 with kdd/Training/model\guess_passwd\guess_passwd_LSTM_model.h5
httptunnel
kdd/Training/model\httptunnel\httptunnel_LSTM_model.h5
Changing kdd/Training/model\httptunnel\LSTM.h5 with kdd/Training/model\httptunnel\httpt

In [12]:
files = glob.glob(f'kdd/Training/model/**', recursive=True)
for file in files:
    if 'LSTM' in file:
        print(file)

kdd/Training/model\apache2\LSTM.h5
kdd/Training/model\back\LSTM.h5
kdd/Training/model\buffer_overflow\LSTM.h5
kdd/Training/model\ftp_write\LSTM.h5
kdd/Training/model\guess_passwd\LSTM.h5
kdd/Training/model\httptunnel\LSTM.h5
kdd/Training/model\imap\LSTM.h5
kdd/Training/model\ipsweep\LSTM.h5
kdd/Training/model\land\LSTM.h5
kdd/Training/model\loadmodule\LSTM.h5
kdd/Training/model\mailbomb\LSTM.h5
kdd/Training/model\mscan\LSTM.h5
kdd/Training/model\multihop\LSTM.h5
kdd/Training/model\named\LSTM.h5
kdd/Training/model\neptune\LSTM.h5
kdd/Training/model\nmap\LSTM.h5
kdd/Training/model\perl\LSTM.h5
kdd/Training/model\phf\LSTM.h5
kdd/Training/model\pod\LSTM.h5
kdd/Training/model\portsweep\LSTM.h5
kdd/Training/model\processtable\LSTM.h5
kdd/Training/model\ps\LSTM.h5
kdd/Training/model\rootkit\LSTM.h5
kdd/Training/model\saint\LSTM.h5
kdd/Training/model\satan\LSTM.h5
kdd/Training/model\sendmail\LSTM.h5
kdd/Training/model\smurf\LSTM.h5


In [14]:
from keras.models import load_model

In [ ]:
model = load_model('kdd/Training/model\land\LSTM.h5')

In [4]:
import pandas as pd
import os
os.getcwd()


'C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\Program'

In [6]:
df = pd.read_csv('./cic/CIC_IDS2017.csv')

In [7]:
df

,source_port,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,...,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,destination_port_priority_1.0,destination_port_priority_2.0,destination_port_priority_3.0
0,0.780453,1.000000,8.411233e-01,0.000014,0.000014,1.100775e-05,1.043553e-06,0.001491,0.014624,0.005976,...,0.000793,7.926000e-04,0.840395,0.000000,0.840395,0.840395,BENIGN,1.0,0.0,0.0
1,0.906157,0.352941,6.986333e-04,0.000041,0.000017,5.527132e-05,7.307923e-07,0.018896,0.000000,0.012002,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,BENIGN,1.0,0.0,0.0
2,0.006760,0.352941,1.333333e-07,0.000005,0.000000,2.868217e-06,0.000000e+00,0.001491,0.000000,0.003114,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,BENIGN,0.0,0.0,1.0
3,0.000000,0.000000,9.891655e-01,0.000341,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.059006,9.090909e-09,0.084367,0.052749,0.140269,0.050029,BENIGN,1.0,0.0,0.0
4,0.906157,0.352941,1.416667e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,BENIGN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827871,0.768750,0.352941,9.695412e-01,0.000100,0.000072,1.392248e-04,1.197035e-05,0.029089,0.000000,0.013144,...,0.008803,2.175909e-04,0.079939,0.018501,0.083333,0.042291,BENIGN,1.0,0.0,0.0
2827872,0.768673,0.352941,9.702959e-01,0.000114,0.000079,8.666667e-05,1.545649e-05,0.013175,0.000000,0.007238,...,0.006509,2.073909e-04,0.080185,0.017020,0.083333,0.045552,BENIGN,1.0,0.0,0.0
2827873,0.768658,0.352941,9.712000e-01,0.000077,0.000051,7.410853e-05,2.285442e-06,0.017446,0.000000,0.008940,...,0.008688,3.327273e-04,0.083319,0.000250,0.083333,0.082859,BENIGN,1.0,0.0,0.0
2827874,0.769009,0.352941,9.973248e-01,0.000086,0.000075,1.796899e-04,1.267673e-05,0.042828,0.000000,0.019509,...,0.029093,8.107455e-04,0.484167,0.003344,0.485833,0.483333,BENIGN,1.0,0.0,0.0


In [10]:
df1 = pd.read_csv('./cic/train_test_folder/train_cic/train.csv')
df2 = pd.read_csv('./cic/train_test_folder/test_cic/test.csv')


In [11]:
print(df1.shape)
print(df2.shape)

(1979513, 84)
(848363, 84)


In [12]:
1979513 + 848363

2827876

In [13]:
df = pd.read_csv('./kdd/KDD.csv')

In [14]:
df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0.0,5.753774e-07,2.542106e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148513,0.0,2.297162e-07,7.160648e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148514,0.0,3.952277e-05,6.346868e-06,0.0,0.0,0.0,0.019802,0.0,1.0,0.000134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148515,0.0,3.043558e-08,3.206260e-08,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
